Implementação do SIMPLEX: 
Entrada esperada: 
    A (np.array) = Coeficientes das restrições
    B (np.array) = Vetor dos resultados de cada igualdade
    C (np.array) = Coeficientes da F.O
    aux (np.array) = Vetor com os sinais ">", "<" referente a cada restrição
    r (String) = String que indica se é maximização ou minimização.  

In [10]:
import numpy as np

In [11]:
def criar_tableau(A, b, c):
    n_vars = len(c)
    n_rest = len(A)
    print(c)
    T = np.zeros((n_rest + 1, n_rest + n_vars + 1))
    for i in range(n_rest): 
        T[i, :n_vars] = A[i]
        T[i, n_vars + i] = 1
        T[i, -1] = b[i]
    T[-1, :n_vars] = c
    return T


In [12]:
def criar_tableau_min(A, b, c, aux):
    n_vars = len(c)
    n_rest = len(A)
    n_sobra = len(aux)
    m = 1000000
    y = (n_rest + n_vars + n_sobra) - n_sobra
    T = np.zeros((n_rest + 1, n_rest + n_vars + n_sobra))
    for i in range(n_rest): 
        T[i, :n_vars] = A[i]
        if aux[i] == "<":
            T[i, n_vars + i] = 1
        elif aux[i] == ">":
            T[i, n_vars + i] = -1
            T[i, y] = 1
            T[-1, y] = m
            y += 1
        T[i, -1] = b[i]
    T[-1, :n_vars] = c
    return T
    

In [13]:
def tratar_tableau_min(T, aux):
    #Achar linhas com valores com variaveis de folga
    m = 1000000
    linhas= []
    for i in range(len(aux)):
        if aux[i] == ">":
            linhas.append(i)
    #Agora temos que multiplicar as linhas por m 
    for i in linhas:
        T[-1] = T[-1] + (T[i] * - m)
    return T

In [14]:
def achar_pivo(T):
    # Encontrando a coluna do pivo
    col_pivo = np.argmin(T[-1, :-1])
    
    # Encontrando a Linha do pivo
    menor = np.inf
    lin_pivo = -1
    for i in range(len(T)):
        if T[i, col_pivo] > 0:
            aux = T[i, -1]/T[i, col_pivo]
            if aux < menor:
                lin_pivo = i
                menor = aux
    
    return lin_pivo, col_pivo

In [15]:
def dividir_linha(T, col_pivo, lin_pivo): 
    pivo = T[lin_pivo, col_pivo]
    for i in range(len(T[0])):
        T[lin_pivo, i] = T[lin_pivo, i] / pivo
    return T

In [16]:
def subtrair_linhas(T, col_pivo, lin_pivo):
    for i in range(T.shape[0]):
        if i != lin_pivo:
            multiplicador = T[i, col_pivo]
            T[i, :] -= multiplicador * T[lin_pivo, :]
    return T

In [17]:
def simplex(A,b,c,aux,r):
    '''
    Entrada: A = Coef. das restrições 
             B = Resultado das restrições
             C = Coef.  das variaveis da F.O
    Saída:  Z = Valor ótimo
            variaveis[] = Coef. ótimo das variaveis 
    '''
    if r == 'max':
        c = -c
        T = criar_tableau(A,b,c)
    else:
        print("Minimização")
        '''
        Para minimizar, temos que trabalhar com um tableau diferente, pois temos que tratar as restrições de maior e menor.
        Para isso, criei ma função que cria um tableau para minimização, e outra que trata esse tableau.
        Nessa implementação, optei pelo método BIG-M, que consiste em multiplicar a linha da função objetivo por um valor muito grande (m = 1000000) 
        '''
        T = criar_tableau_min(A,b,c,aux)
        T = tratar_tableau_min(T,aux)
     #Primeiro passo do simplex, dado a F.O e as restrições, criar uma tabela (tableau) para ser manipulada.
    while np.min(T[-1, :-1]) < 0:
        '''
        A grosso modo, o simplex é um método interativo que visa investigar os pontos extremos de uma função cortada pelas suas restrições.
        Com isso, temos um loop que se repete até que não restem valores negativos na linha da função objetivo:
            Achamos o pivo
            Dividimos a linha por esse pivo
            Subtraimos as linhas restantes a fim de zerar a coluna abaixo do pivo
        '''
        lin_pivo, col_pivo = achar_pivo(T)
        T = dividir_linha(T, col_pivo, lin_pivo)
        T = subtrair_linhas(T, col_pivo, lin_pivo)
    Z = T[-1,-1]
    if r == 'min':
        Z = -Z
    variaveis = []
    for i in range(len(c)):
        for j in range(len(T)):
            if T[j,i] == 1:
                variaveis.append(T[j,-1])
                break
    
    return Z, variaveis

In [18]:
A = np.array([[5, -5],[3, 1]])
b = np.array([10, 12])
c = np.array([1,2])
aux = np.array([">", "<"])
r = 'min'
Z, variaveis = simplex(A,b,c,aux,r)
print("Z = ", Z)
print("Variaveis = ", variaveis)


Minimização
Z =  2.0
Variaveis =  [2.0]
